## Check point
- 학습을 하다가 weight 를 저장함 -> 다음에 학습을 할때 이어갈 수 있음

In [2]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt
%matplotlib inline

import math

## Hyperparameter Tunning

In [3]:
num_epochs = 20
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [4]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [5]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

## Data Preprocess

In [36]:
train_paths = glob('../dataset/cifar/train/*.png')[:1000]
test_paths = glob('../dataset/cifar/test/*.png')[:1000]

In [37]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [38]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [39]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [40]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [41]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [42]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [43]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [44]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

## Checkpoint

In [51]:
save_path = 'checkpoints'
# moitor: 기준을 val_accuracy, val_loss, accuracy, loss로도 잡을수 있으며, 
# save_best_only: accuracy가 올라가면 저장하라
# mode: accuray일때는 max loss일때는 min
# verbose: 저장할때 로그로 보일지 여부

check_point = tf.keras.callbacks.ModelCheckpoint(save_path, moitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

## Training
- http://localhost:6006

In [52]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[check_point]
)

Epoch 1/20
30/31 [============================>.] - ETA: 0s - loss: 1.2290 - accuracy: 0.5406
Epoch 00001: val_loss improved from -inf to 1.68153, saving model to checkpoints
INFO:tensorflow:Assets written to: checkpoints\assets
31/31 [==============================] - 9s 274ms/step - loss: 1.2252 - accuracy: 0.5434 - val_loss: 1.6815 - val_accuracy: 0.4123
Epoch 2/20
30/31 [============================>.] - ETA: 0s - loss: 1.1447 - accuracy: 0.5962
Epoch 00002: val_loss improved from 1.68153 to 1.69121, saving model to checkpoints
INFO:tensorflow:Assets written to: checkpoints\assets
31/31 [==============================] - 9s 303ms/step - loss: 1.1553 - accuracy: 0.5950 - val_loss: 1.6912 - val_accuracy: 0.4143
Epoch 3/20
30/31 [============================>.] - ETA: 0s - loss: 1.1238 - accuracy: 0.6004
Epoch 00003: val_loss improved from 1.69121 to 1.78030, saving model to checkpoints
INFO:tensorflow:Assets written to: checkpoints\assets
31/31 [==============================] - 8s 2